In [5]:
import numpy as np 
import matplotlib.pyplot as plt
import stumpy
import pandas as pd
from scipy import signal
from sklearn.linear_model import Ridge
from scipy.spatial import distance
from fatf.utils.kernels import exponential_kernel 

import warnings
warnings.filterwarnings('ignore')

import importlib
import sys
sys.path.append('../')
from Utils.explanationsnew import LIMESegment, NEVES, LEFTIST, NNSegment, RBP, background_perturb
from Utils.data import loadUCRDataID
from Utils.models import *
from Utils.metrics import *

In [6]:
dataset_map = [('Coffee', 0),
                ('Strawberry', 1),
                   ('GunPointOldVersusYoung', 2),
                   ('HandOutlines', 3),
                    ('yoga', 4),
                    ('ECG200', 5),
                    ('GunPointMaleVersusFemale', 6),
                    ('DodgerLoopGame', 7),
                    ('Chinatown', 8),
                    ('FreezerSmallTrain', 9),
                    ('HouseTwenty', 10),
                    ('WormsTwoClass', 11)
                    ]

In [13]:
datasets = {}
for data_idx in dataset_map:
    datasets[data_idx[0]] = loadUCRDataID(data_idx[1])

Loading train / test dataset :  ../Data/Coffee_TRAIN ../Data/Coffee_TEST
Loading train / test dataset :  ../Data/Strawberry_TRAIN ../Data/Strawberry_TEST
Loading train / test dataset :  ../Data/GunPointOldVersusYoung_TRAIN ../Data/GunPointOldVersusYoung_TEST
Loading train / test dataset :  ../Data/HandOutlines_TRAIN ../Data/HandOutlines_TEST
Loading train / test dataset :  ../Data/Yoga_TRAIN ../Data/Yoga_TEST
Loading train / test dataset :  ../Data/ECG200_TRAIN ../Data/ECG200_TEST
Loading train / test dataset :  ../Data/GunPointMaleVersusFemale_TRAIN ../Data/GunPointMaleVersusFemale_TEST
Loading train / test dataset :  ../Data/DodgerLoopGame_TRAIN ../Data/DodgerLoopGame_TEST
Loading train / test dataset :  ../Data/Chinatown_TRAIN ../Data/Chinatown_TEST
Loading train / test dataset :  ../Data/FreezerSmallTrain_TRAIN ../Data/FreezerSmallTrain_TEST
Loading train / test dataset :  ../Data/HouseTwenty_TRAIN ../Data/HouseTwenty_TEST
Loading train / test dataset :  ../Data/WormsTwoClass_TRAIN

In [4]:
"""Datasets['Coffee'] : Train x, Train y, Test x, Test y

Train x : N x T x 1
Train y: N x 1 
Test x: N' x T x 1
Test y: N' x 2"""

"Datasets['Coffee'] : Train x, Train y, Test x, Test y\n\nTrain x : N x T x 1\nTrain y: N x 1 \nTest x: N' x T x 1\nTest y: N' x 2"

In [24]:
for data_idx in datasets.keys():
    print(datasets[data_idx][0].shape)

(28, 286, 1)
(613, 235, 1)
(136, 150, 1)
(1000, 2709, 1)
(300, 426, 1)
(100, 96, 1)
(135, 150, 1)
(20, 288, 1)
(20, 24, 1)
(28, 301, 1)
(40, 2000, 1)
(181, 900, 1)


In [16]:
BATCH_SIZES = [8, 64, 32, 64, 64, 16, 16, 4, 4, 4, 8, 16]
WINDOW_SIZES = [20, 20, 10, 100, 50, 10, 10, 20, 3, 10, 200, 100]
CPS = [5, 5, 4, 8, 5, 3, 4, 5, 2, 5, 8, 6]

In [17]:
models = ['knn','cnn','lstmfcn']
trained_models = {}
i = 0
for data_idx in datasets.keys():
    print(data_idx)
    trained_models[data_idx] = {}
    trained_models[data_idx]['knn'] = train_KNN_model(datasets[data_idx][0],datasets[data_idx][1])
    model_cnn = make_CNN_model(datasets[data_idx][0].shape[1:])
    trained_models[data_idx]['cnn'] = train_CNN_model(model_cnn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])[0]
    model_lstmfcn = make_LSTMFCN_model(datasets[data_idx][0].shape[1])
    trained_models[data_idx]['lstmfcn'] = train_LSTMFCN_model(model_lstmfcn,
                                                      datasets[data_idx][0],
                                                      datasets[data_idx][1],
                                                      datasets[data_idx][2],
                                                      datasets[data_idx][3],
                                                      epochs=2,
                                                      batch_size=BATCH_SIZES[i])
    i = i + 1

Coffee
Epoch 1/2


2024-11-04 21:02:57.640963: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


3/3 [==============================] - 0s 68ms/step - loss: 0.7179 - sparse_categorical_accuracy: 0.2727 - val_loss: 0.6727 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/2
3/3 [==============================] - 0s 13ms/step - loss: 0.6565 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.6862 - val_sparse_categorical_accuracy: 1.0000
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 286, 1)]     0           []                               
                                                                                                  
 permute (Permute)              (None, 1, 286)       0           ['input_2[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 1, 128) 

2024-11-04 21:02:58.272665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-11-04 21:02:58.273222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-11-04 21:02:58.273579: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/2


...

16/16 - 20s - loss: 0.4581 - accuracy: 0.8260 - val_loss: 0.9239 - val_accuracy: 0.3595
Epoch 2/2
16/16 - 17s - loss: 0.3096 - accuracy: 0.8830 - val_loss: 0.6961 - val_accuracy: 0.4649
yoga
Epoch 1/2
4/4 [==============================] - 2s 225ms/step - loss: 0.8689 - sparse_categorical_accuracy: 0.4750 - val_loss: 0.6953 - val_sparse_categorical_accuracy: 0.3833
Epoch 2/2
4/4 [==============================] - 1s 155ms/step - loss: 0.7212 - sparse_categorical_accuracy: 0.4792 - val_loss: 0.6856 - val_sparse_categorical_accuracy: 0.6167
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 426, 1)]     0                                            
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 1, 

7/7 - 4s - loss: 0.4450 - accuracy: 0.7800 - val_loss: 0.6286 - val_accuracy: 0.8600
Epoch 2/2
7/7 - 0s - loss: 0.2656 - accuracy: 0.8500 - val_loss: 0.5805 - val_accuracy: 0.8200
GunPointMaleVersusFemale
Epoch 1/2
7/7 [==============================] - 1s 52ms/step - loss: 0.4741 - sparse_categorical_accuracy: 0.7963 - val_loss: 16.3325 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
7/7 [==============================] - 0s 22ms/step - loss: 0.3933 - sparse_categorical_accuracy: 0.8333 - val_loss: 16.5262 - val_sparse_categorical_accuracy: 0.0000e+00
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 150, 1)]     0                                            
__________________________________________________________________________________________________
permute_6 (Permute)

5/5 - 4s - loss: 0.8542 - accuracy: 0.6000 - val_loss: 3.2380 - val_accuracy: 0.4783
Epoch 2/2
5/5 - 1s - loss: 0.4102 - accuracy: 0.8000 - val_loss: 5.4377 - val_accuracy: 0.4783
Chinatown
Epoch 1/2
4/4 [==============================] - 1s 77ms/step - loss: 0.6820 - sparse_categorical_accuracy: 0.5625 - val_loss: 1.7014 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
4/4 [==============================] - 0s 10ms/step - loss: 0.4849 - sparse_categorical_accuracy: 0.8125 - val_loss: 2.6523 - val_sparse_categorical_accuracy: 0.0000e+00
Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           [(None, 24, 1)]      0                                            
__________________________________________________________________________________________________
permute_8 (Permute)             (Non

7/7 - 15s - loss: 0.8649 - accuracy: 0.6071 - val_loss: 0.6185 - val_accuracy: 0.7323
Epoch 2/2
7/7 - 11s - loss: 0.7939 - accuracy: 0.6786 - val_loss: 0.6510 - val_accuracy: 0.6326
HouseTwenty
Epoch 1/2
4/4 [==============================] - 1s 164ms/step - loss: 0.6275 - sparse_categorical_accuracy: 0.5938 - val_loss: 1.3767 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/2
4/4 [==============================] - 0s 114ms/step - loss: 0.4769 - sparse_categorical_accuracy: 0.8438 - val_loss: 1.1808 - val_sparse_categorical_accuracy: 0.0000e+00
Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 2000, 1)]    0                                            
__________________________________________________________________________________________________
permute_10 (Permute)          

12/12 - 6s - loss: 0.9019 - accuracy: 0.5193 - val_loss: 0.6786 - val_accuracy: 0.5584
Epoch 2/2
12/12 - 3s - loss: 0.5246 - accuracy: 0.7348 - val_loss: 0.7106 - val_accuracy: 0.5584


In [8]:
"""model_KNN shape : N x T 
## model_CNN/LSTMFCN shape  : N x T x 1


SyntaxError: EOF while scanning triple-quoted string literal (3663073547.py, line 2)

In [18]:
MODEL_TYPES = ['class','proba','proba']

In [21]:
def reshaper(x,j):
    if j == 0:
        return x.reshape(x.shape[0])
    else:
        return x
reshaper(datasets[data_idx][2][0:2][0],0)

array([-0.7785892 , -0.74443593, -0.72546189, -0.70269304, -0.71407746,
       -0.72166708, -0.73305151, -0.71787227, -0.73684631, -0.76340997,
       -0.77479439, -0.76340997, -0.74823074, -0.72546189, -0.73305151,
       -0.73305151, -0.74064112, -0.74443593, -0.76340997, -0.76720478,
       -0.76340997, -0.75582035, -0.74823074, -0.74823074, -0.75582035,
       -0.74823074, -0.75961516, -0.75202555, -0.74823074, -0.74823074,
       -0.73305151, -0.73305151, -0.74064112, -0.77099959, -0.80894767,
       -0.83930613, -0.86586979, -0.87725421, -0.8734594 , -0.9189971 ,
       -0.86207498, -0.86207498, -0.85069055, -0.82792171, -0.85069055,
       -0.78238401, -0.76720478, -0.74443593, -0.72546189, -0.75202555,
       -0.69510342, -0.69510342, -0.72166708, -0.7292567 , -0.74443593,
       -0.74443593, -0.79756324, -0.75202555, -0.67233458, -0.64577092,
       -0.67612938, -0.65715534, -0.61541246, -0.6381813 , -0.6343865 ,
       -0.59264361, -0.61161765, -0.55469553, -0.42567205, -0.30

In [13]:
explanations = {}
i = 0 
noisy_explanations = {} # For Robustness later
for data_idx in datasets.keys():
    print('processing explanations for: ' + str(data_idx) + '\n')
    explanations[data_idx] = {}
    noisy_explanations[data_idx] = {}
    j = 0
    for model_idx in trained_models[data_idx].keys():
        print('processing explanations for: ' + str(model_idx) + '\n')
        explanations[data_idx][model_idx] = {}
        noisy_explanations[data_idx][model_idx] = {}
        explanation_set = datasets[data_idx][2][0:2]
        explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in explanation_set]
        explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in explanation_set]
        explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in explanation_set]
        
        noisy_set = np.asarray([add_noise(x) for x in explanation_set])
        
        noisy_explanations[data_idx][model_idx]['LS'] = [LIMESegment(reshaper(x,j), trained_models[data_idx][model_idx], model_type=MODEL_TYPES[j], window_size=WINDOW_SIZES[i], cp=CPS[i]) for x in noisy_set]
        noisy_explanations[data_idx][model_idx]['N'] = [NEVES(x, trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j]) for x in noisy_set]
        noisy_explanations[data_idx][model_idx]['LF'] = [LEFTIST(reshaper(x,j), trained_models[data_idx][model_idx], datasets[data_idx][0], model_type=MODEL_TYPES[j],) for x in noisy_set]
        
        j = j + 1
    i = i + 1

processing explanations for: Coffee

processing explanations for: knn



KeyboardInterrupt: 

In [ ]:
evaluation_metrics = {}
for data_idx in datasets.keys():
    evaluation_metrics[data_idx] = {}
    j = 0 
    for model_idx in trained_models[data_idx].keys():
        evaluation_metrics[data_idx][model_idx] = {}
        for explanation_idx in explanations[data_idx][model_idx].keys():
            evaluation_metrics[data_idx][model_idx][explanation_idx] = {}
            # Robustness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Robustness'] = robustnessy(explanations[data_idx][model_idx][explanation_idx],
                                                                                         noisy_explanations[data_idx][model_idx][explanation_idx])
            explanation_set = datasets[data_idx][2][0:2]
            explanation_labels = datasets[data_idx][3][0:2]
            if j == 0:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set.reshape(explanation_set.shape[:2]))
            
            else:
                explanation_predictions = trained_models[data_idx][model_idx].predict(explanation_set)
                
                # Faithfulness
            evaluation_metrics[data_idx][model_idx][explanation_idx]['Faithfulness'] = faithfulnessy(explanations[data_idx][model_idx][explanation_idx],explanation_set,explanation_labels,explanation_predictions,trained_models[data_idx][model_idx],model_type=MODEL_TYPES[j])
        j+=1
    